<a href="https://colab.research.google.com/github/orenodinner/PConv-Keras-for-google-colab/blob/master/4_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

アップロードコード

In [1]:
!git clone "https://github.com/orenodinner/PConv-Keras-for-google-colab.git"

Cloning into 'PConv-Keras-for-google-colab'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 296 (delta 10), reused 0 (delta 0), pack-reused 277
Receiving objects: 100% (296/296), 52.10 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [2]:
%cd PConv-Keras-for-google-colab

/content/PConv-Keras-for-google-colab


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
from oauth2client.client import GoogleCredentials
import getpass
auth.authenticate_user()
creds = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

xxxxx.zipのようにzipで圧縮してGoogle Driveに普通にアップロードする。
Colab上で以下を実行。/content/dataというディレクトリに大量データが展開される。
ポイントは、> /dev/null 2>&1 &で、標準出力を捨てているところだ。

そうしないと、unzipで大量データを展開したときに大量の標準出力が表示されてしまい、Chromeがクラッシュして落ちてしまうことがある。

In [0]:
!mkdir /content/PConv-Keras-for-google-colab/data_img/
!mkdir /content/PConv-Keras-for-google-colab/data_img/images
!unzip /content/PConv-Keras-for-google-colab/drive/resize.zip  -d /content/PConv-Keras-for-google-colab/data_img/images > /dev/null 2>&1 &

解凍後のファイル数を確認するには以下を実行する。

In [11]:
!echo ./data_img/images/* | xargs ls | wc

   1094    1094   29538


In [12]:
import gc
import datetime

import pandas as pd
import numpy as np

from copy import deepcopy
from tqdm import tqdm

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K

import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from IPython.display import clear_output

from libs.pconv_model import PConvUnet
from libs.util import random_mask

%load_ext autoreload
%autoreload 2
plt.ioff()

# SETTINGS
TRAIN_DIR = r"./data_img/"  #要編集
TEST_DIR = r"./data_img/"
VAL_DIR = r"./data_img/"

BATCH_SIZE = 4

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:

# SETTINGS
TRAIN_DIR = r"./data_img/"  #要編集
TEST_DIR = r"./data_img/"
VAL_DIR = r"./data_img/"

BATCH_SIZE = 4

In [0]:
class DataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, *args, **kwargs):
        generator = super().flow_from_directory(directory, class_mode=None, *args, **kwargs)
        while True:
            
            # Get augmentend image samples
            ori = next(generator)

            # Get masks for each image sample
            mask = np.stack([random_mask(ori.shape[1], ori.shape[2]) for _ in range(ori.shape[0])], axis=0)

            # Apply masks to all image sample
            masked = deepcopy(ori)
            masked[mask==0] = 1

            # Yield ([ori, masl],  ori) training batches
            # print(masked.shape, ori.shape)
            gc.collect()
            yield [masked, mask], ori
            
# Create training generator
train_datagen = DataGenerator(  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(512, 512), batch_size=BATCH_SIZE
)

# Create validation generator
val_datagen = DataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    VAL_DIR, target_size=(512, 512), batch_size=BATCH_SIZE, seed=1
)

# Create testing generator
test_datagen = DataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR, target_size=(512, 512), batch_size=BATCH_SIZE, seed=1
)

In [15]:
# Pick out an example
test_data = next(test_generator)
(masked, mask), ori = test_data

# Show side by side
for i in range(len(ori)):
    _, axes = plt.subplots(1, 3, figsize=(20, 5))
    axes[0].imshow(masked[i,:,:,:])
    axes[1].imshow(mask[i,:,:,:] * 1.)
    axes[2].imshow(ori[i,:,:,:])
    #plt.show()

Found 1094 images belonging to 1 classes.


In [0]:
def plot_callback(model):
    """Called at the end of each epoch, displaying our previous test images,
    as well as their masked predictions and saving them to disk"""
    
    # Get samples & Display them        
    pred_img = model.predict([masked, mask])
    pred_time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

    # Clear current output and display test images
    for i in range(len(ori)):
        _, axes = plt.subplots(1, 3, figsize=(20, 5))
        axes[0].imshow(masked[i,:,:,:])
        axes[1].imshow(pred_img[i,:,:,:] * 1.)
        axes[2].imshow(ori[i,:,:,:])
        axes[0].set_title('Masked Image')
        axes[1].set_title('Predicted Image')
        axes[2].set_title('Original Image')
                
        plt.savefig(r"./data/img_{}_{}.png".format(i, pred_time))
        plt.close()
        #plt.show()#ore追記

In [0]:
# Instantiate the model
model = PConvUnet(weight_filepath='./data/logs/')
# model.load(r"./data/logs/50_weights_2018-06-01-16-41-43.h5")

In [0]:
# Run training for certain amount of epochs
model.fit(
    train_generator, 
    steps_per_epoch=1000,
    validation_data=val_generator,
    validation_steps=100,
    epochs=50,        
    plot_callback=plot_callback,
    callbacks=[
        TensorBoard(log_dir='./data/logs/initial_training', write_graph=False)
    ]
)

Epoch 1/1
 650/1000 [==================>...........] - ETA: 14:31 - loss: 920597.0121

In [0]:
# Load weights from previous run
model = PConvUnet(weight_filepath='data/logs/')
model.load(
    r"./data/logs/50_weights_2019-01-31-02-35-12.h5", #修正必要
    train_bn=False,
    lr=0.00005
)

In [0]:
# Run training for certain amount of epochs
model.fit(
    train_generator, 
    steps_per_epoch=10,
    validation_data=val_generator,
    validation_steps=10,
    epochs=50,        
    workers=3,
    plot_callback=plot_callback,
    callbacks=[
        TensorBoard(log_dir='./data/logs/fine_tuning', write_graph=False)
    ]
)

リザルト

In [0]:
# Load weights from previous run
model = PConvUnet(weight_filepath='./data/logs/')
model.load(
    r"./data/logs/50_weights_2019-01-31-02-35-12.h5",
    train_bn=False,
    lr=0.00005
)

In [0]:
mkdir ./kekka_jpg

In [0]:
n = 0
for (masked, mask), ori in tqdm(test_generator):
    
    # Run predictions for this batch of images
    pred_img = model.predict([masked, mask])
    pred_time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    
    # Clear current output and display test images
    for i in range(len(ori)):
        _, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(masked[i,:,:,:])
        axes[1].imshow(pred_img[i,:,:,:] * 1.)
        axes[0].set_title('Masked Image')
        axes[1].set_title('Predicted Image')
        axes[0].xaxis.set_major_formatter(NullFormatter())
        axes[0].yaxis.set_major_formatter(NullFormatter())
        axes[1].xaxis.set_major_formatter(NullFormatter())
        axes[1].yaxis.set_major_formatter(NullFormatter())
                
        plt.savefig(r'./kekka_jpg/img_{}_{}.png'.format(i, pred_time))
        plt.close()
       # plt.show()
        n += 1
        
    # Only create predictions for about 100 images
    if n > 100:
        break